In [ ]:
!pip install skorch

In [ ]:
import os
import glob
import tqdm
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import skorch
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, roc_auc_score

# SAT-6 Convolutional Neural Network (20 pts.)

Reference: Saikat Basu, Sangram Ganguly, Supratik Mukhopadhyay, Robert Dibiano, Manohar Karki and Ramakrishna Nemani, [DeepSat - A Learning framework for Satellite Imagery](https://arxiv.org/abs/1509.03602), ACM SIGSPATIAL 2015.

<img src='https://csc.lsu.edu/~saikat/deepsat/images/sat_img.png'>

SAT-6 consists of a total of 405,000 image patches each of size 28x28 and covering 6 landcover classes - barren land, trees, grassland, roads, buildings and water bodies. 324,000 images (comprising of four-fifths of the total dataset) were chosen as the training dataset and 81,000 (one fifths) were chosen as the testing dataset. The training and test sets were selected from disjoint NAIP tiles. Once generated, the images in the dataset were randomized.

The MAT file for the SAT-6 dataset contains the following variables:

- `train_x`: 28x28x4x324000 uint8 (containing 324000 training samples of 28x28 images each with 4 channels)
- `train_y`: 324000x6 uint8 (containing 6x1 vectors having labels for the 324000 training samples)
- `test_x`: 28x28x4x81000 uint8 (containing 81000 test samples of 28x28 images each with 4 channels)
- `test_y`: 81000x6 uint8 (containing 6x1 vectors having labels for the 81000 test samples) 




### Download the dataset

In [ ]:
!gdown --id 0B0Fef71_vt3PUkZ4YVZ5WWNvZWs

In [ ]:
os.system('tar -xvzf SAT-4_and_SAT-6_datasets.tar.gz')

### Load the dataset

In [ ]:
def load_sat6_dataset():

  from scipy.io import loadmat
  sat6 = loadmat('sat-6-full.mat')

  preprocess = transforms.Compose([transforms.ToTensor(),
                                   transforms.Resize((64, 64)),
                                   # normalization used on training resnet-50 data
                                   transforms.Normalize(mean=[0.7137, 0.6628, 0.6519], \
                                                        std=[0.2970, 0.3017, 0.2979]),])

  X_train, X_test, y_train, y_test = sat6['train_x'], sat6['test_x'], sat6['train_y'], sat6['test_y']
  X_train = np.transpose(X_train, axes=[3,0,1,2])[:80000]
  X_train = np.stack([preprocess(X_train[i, :, :, :3]) for i in range(X_train.shape[0])], axis=0)
  X_test = np.transpose(X_test, axes=[3,0,1,2])[:20000]
  X_test = np.stack([preprocess(X_test[i, :, :, :3]) for i in range(X_test.shape[0])], axis=0)
  y_train = np.argmax(y_train, axis=0)[:80000]
  y_test = np.argmax(y_test, axis=0)[:20000]
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_sat6_dataset()

## Question 1: Train a convolutional neural network for land use classification (10 pts)
Use a pre-trained neural network from `torchvision.models` to fine-tune a land-use classifier using the SAT-6 dataset.

## Question 2: Train a convolutional neural network for land use classification with data augmentation (10 pts)
Try to improve the performance by adding data augmentation techniques to the `load_sat6_dataset` function.